In [35]:
# general imports
import pandas as pd
import re

In [43]:
# create a dataframe of tweets from LA Times Health and ignore the id and timestamp columns
tweets_df = pd.read_csv('https://raw.githubusercontent.com/nikhilmanda9/Tweets-Similarity/main/latimeshealth.txt', delimiter='|', names=('id', 'timestamp', 'tweet'), usecols=['tweet'],header=None)

# convert the dataframe to a nested list of tweets
tweets = tweets_df.values.tolist()

In [45]:
# regular expression to extract URL
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

# regular expression to extract '@' pattern
mention_pattern = re.compile(r'@(\w+)')

# regular expression to extract '#' pattern
hashtag_pattern = re.compile(r'#(\w+)')


for k in range(len(tweets)):
  # remove the URL from the tweet
  tweets[k][0] = url_pattern.sub('', tweets[k][0])

  # remove the '@' character from the tweet
  tweets[k][0] = mention_pattern.sub(r'\1', tweets[k][0])

  # remove the '#' character from the tweet
  tweets[k][0] = hashtag_pattern.sub(r'\1', tweets[k][0])

  # remove trailing whitespace at the end of the tweet
  tweets[k][0] = tweets[k][0].strip()

  # ensure all words in the tweet are lowercase
  tweets[k][0] = tweets[k][0].lower()